In [1]:
import os
import skimage

import numpy as np
import pandas as pd
import skimage.io as io


from PIL import Image
from typing import List, Any, Tuple
from skimage.restoration import  denoise_nl_means
from skimage.metrics import peak_signal_noise_ratio

import torchvision.transforms.functional as ft
from torchvision.transforms import Compose, ToTensor, Resize

In [2]:
def nlm_denoising(img:Any) -> Any:
    """ this function takes noisy image and return a denoised version of the input image 

    Args:
        img (Any): input image

    Returns:
        output image
    """    
    img= np.array(img)   
    img = skimage.img_as_float(img)
    nlm_den = denoise_nl_means(img,patch_size=7,patch_distance=11,h=0.1)
    img = skimage.img_as_ubyte(img)
    img = Image.fromarray(img)
    return nlm_den


In [3]:
def resizing(img:Any , H:int, W:int) -> Any:
    """this function takse input image of any extension and return a resized version 
    of it according the specified width and hight

    Args:
        img (Any): the input image
        H (int): the hight of the resized image
        W (int): the width of the resized image

    Returns:
        the resized image
    """    
    img = ft.resize(img,size=[H,W])
    return img



In [4]:
def hflip_aug(img:Any) -> Any:
    """ this function takes input image, perform horizontal flipping and return it


    Args:
        img (Any): the input image

    Returns:
        Any: the output image
    """
    img = ft.hflip(img)    
    return img


In [5]:
def randrot_aug(img: Any, direction: str = 'positive' ) -> Any:
    """this function takes input image, perform random rotation and return it.

    Args:
        img (Any): [description]
        direction (str, optional): the rotation direction. Defaults to 'positive'.

    Returns:
        the reotated image
    """    
    angle = np.random.randint(4,8)

    if direction == 'negative':
        angle *= -1

    img = ft.rotate(img,angle,fill=255)
    return img


In [6]:
def read_iamge(img_dir: str) -> Any:
    """this function read an image from the supplied path

    Args:
        img_dir (str): path to the input image

    Returns:
        PIL Image
    """                
    img = Image.open(img_dir)
    return img


In [7]:
def save_image(img:Any, output_dir: str, img_name: str ):
    """this function convert the input image into uint8 class and save it to a spicific directory

    Args:
        img (Any): input image of type
        output_dir (str): the location at which the image will be saved
        img_name (str): the ultimate name of the image in the form image.format 
    """    
    img.save(fp=os.path.join(output_dir,img_name))


In [ ]:
# Data directories definition
raw_dir = os.path.join(os.curdir,'data','raw','train')
intermed_dir = os.path.join(os.curdir,'data','intermediate')
preprocessed_dir = os.path.join(os.curdir,'data','preprocessed')

In [ ]:
# 1- Images denoising process
for dir in os.listdir(raw_dir):

    image_dir = os.path.join(raw_dir,dir)

    saving_dir = os.path.join(intermed_dir,dir)
    
    for image in os.listdir(image_dir):

        img = read_iamge(os.path.join(image_dir,image))
        img = nlm_denoising(img)
        save_image(img,saving_dir,image,'jpg')

In [ ]:

# 2- image resizing
for dir in os.listdir(intermed_dir):

    image_dir = os.path.join(intermed_dir,dir)
    
    saving_dir = os.path.join(preprocessed_dir,dir)
    
    for image in os.listdir(image_dir):
        
        img = read_iamge(os.path.join(image_dir,image))
        img = resizing(img,224,224)
        save_image(img,saving_dir,image)

In [ ]:
# 3-images augmentation
augmentations = ['h-flip','pos_rot', 'neg_rot']

for augment in augmentations:

    for dir in os.listdir(intermed_dir):

        if dir in ['DME','DRUSEN']:

            image_dir = os.path.join(intermed_dir,dir)
    
            saving_dir = os.path.join(preprocessed_dir,dir)
    
            for image in os.listdir(image_dir):
        
                img = read_iamge(os.path.join(image_dir,image))

                if augment == 'h-flip':
                    img = hflip_aug(img)
                    image = image[:-5] + '-hf' + image[-5:]
                
                if augment == 'pos_rot':
                    img = randrot_aug(img, direction='positive')
                    image = image[:-5] + '-pr' + image[-5:]

                if augment == 'neg_rot':
                    img = randrot_aug(img,direction= 'negative')
                    image = image[:-5] + '-nr' + image[-5:]
                

                img = resizing(img,224,224)
                save_image(img,saving_dir,image)